# HASI Mouse Femur MicroCT Metaphysis Morphometry Features

In [1]:
from urllib.request import urlretrieve
import os
from pathlib import Path

import itk
import numpy as np

from itkwidgets import view, cm
from ipywidgets import FloatProgress, Label, HBox, VBox, FloatText, ColorPicker, Button, Select, Text, Tab

In [2]:
# I/O configuration

inputs_images_dir = Path('input_images')
landmarks_dir = Path('landmarks')
if not landmarks_dir.exists():
    landmarks_dir.mkdir()
output_images_dir = Path('output_images')
if not output_images_dir.exists():
    output_images_dir.mkdir()
output_features_dir = Path('output_features')
if not output_features_dir.exists():
    output_features_dir.mkdir()
atlas_dir = Path('atlas')
if not atlas_dir.exists():
    atlas_dir.mkdir()

In [3]:
# Viewer configuration

camera = np.array([[-8.23076   , 16.761303  , 41.647186  ],
       [ 4.855     ,  9.24      , 48.065998  ],
       [ 0.3404956 , -0.19294305, -0.92023677]], dtype=np.float32)
opacity_gaussians = [[{'position': 0.23888888888888893,
   'height': 0.3090909090909091,
   'width': 0.1722222222222221,
   'xBias': 0.04708953460902432,
   'yBias': 0}]]
cmap = cm.bone
gradient_opacity = 1.0
background = (0.6,0.6,0.6)
ui_collapsed = True

viewer_kwargs = { 'camera': camera,
                  'opacity_gaussians': opacity_gaussians,
                  'cmap': cmap,
                  'gradient_opacity': gradient_opacity,
                  'background': background,
                  'ui_collapsed': ui_collapsed }

In [4]:
# Select input tab

selected_input_path = None
selected_input_image = None
selected_input_callbacks = []

input_images_paths = [str(ii) for ii in inputs_images_dir.glob('*')]
input_images_paths.sort()
selected_input_path = input_images_paths[0]
selected_input_image = itk.imread(selected_input_path)
input_select_widget = Select(options=input_images_paths, value=selected_input_path)

input_viewer = view(image=selected_input_image, **viewer_kwargs)

def on_input_select(change):
    selected_input_path = change.new
    image = itk.imread(change.new)
    selected_input_image = image
    input_viewer.image = image
    for callback in selected_input_callbacks:
        callback(change)
    # input_viewer.reset_roi()
    
input_select_widget.observe(on_input_select, ['value'])
input_select_box = HBox([input_select_widget, input_viewer])
# input_select_box

In [5]:
# Segment bones tab

segment_postfix = '-bones.nrrd'
segment_bones_label = Label('Segment bones')
segment_bones_progress = FloatProgress(value=0.0,
                         min=0.0,
                         max=1.0,
                         step=0.001,
                         description='progress:',
                         bar_style='info')
segment_bones_hbox = HBox([segment_bones_label, segment_bones_progress])

segment_bones_path = None
output_path = output_images_dir / Path(Path(selected_input_path).name + segment_postfix)

segment_bones_viewer = None
if output_path.exists():
    segment_bones_path = str(output_path)
    label_image = itk.imread(str(output_path))
    segment_bones_viewer = view(image=selected_input_image, label_image=label_image, **viewer_kwargs)
else:
    segment_bones_viewer = view(image=selected_input_image, **viewer_kwargs)
segment_bones_vbox = VBox([segment_bones_hbox, segment_bones_viewer])

def on_select_input_segment_bones(change):

    output_path = output_images_dir / Path(Path(selected_input_path).name + segment_postfix)
    segment_bones_path = str(output_path)
    image = selected_input_image
    segment_bones_viewer.image = image
    if output_path.exists():
        label_image = itk.imread(str(output_path))
        segment_bones_viewer.label_image = label_image
        return

    bone_segmenter = itk.SegmentBonesInMicroCTFilter.New(image)
    cortical_thickness=0.1
    bone_segmenter.SetCorticalBoneThickness(cortical_thickness)

    segment_bones_progress.value = 0.0
    segment_bones_progress.bar_style = 'info'
    def update_progress():
        segment_bones_progress.value = bone_segmenter.GetProgress()
    bone_segmenter.AddObserver(itk.ProgressEvent(), update_progress)
    bone_segmenter.Update()
    
    segment_bones_path = str(output_path)
    itk.imwrite(bone_segmenter.GetOutput(), segment_bones_path)

    segment_bones_progress.bar_style = 'success'
    segment_bones_progress.description = 'done.'
    
    segment_bones_viewer.label_image=bone_segmenter.GetOutput()
    
selected_input_callbacks.append(on_select_input_segment_bones)

In [6]:
# Select landmarks tab
 
atlas_path = atlas_dir / Path('907-L.nrrd')
if not atlas_path.exists():

    url = 'https://data.kitware.com/api/v1/file/5ef375199014a6d84edb6a1f/download'
    urlretrieve(url, str(atlas_path))

atlas = itk.imread(str(atlas_path))

atlas_label_path = atlas_dir / Path('907-L-atlas.nrrd')
if not atlas_label_path.exists():
    url = 'https://data.kitware.com/api/v1/file/5ef372559014a6d84edb627e/download'
    urlretrieve(url, str(atlas_label_path))

atlas_label = itk.imread(str(atlas_label_path))


center_of_femur_head = np.array([4.072,11.347,43.816]).reshape((1,3))
femur_shaft = np.array([7.713,9.107,42.649]).reshape((1,3))
dent = np.array([3.787,11.172,45.173]).reshape((1,3))

opacity_gaussians_landmark = [[{'position': 0.33611111111111114,
   'height': 0.24545454545454548,
   'width': 0.22499999999999998,
   'xBias': 0.51,
   'yBias': 0.4}]]
viewer_atlas = view(atlas,
     #label_image=atlas_label,
     gradient_opacity=gradient_opacity,
     background=background,
     cmap=cmap,
     camera=camera,
     opacity_gaussians=opacity_gaussians_landmark,
     point_set_sizes=[15,]*3,
     ui_collapsed=ui_collapsed,
     point_set_representations=['points',]*3,
     point_sets=[center_of_femur_head, femur_shaft, dent])

center_of_femur_head = np.array([4.945,9.225,46.011]).reshape((1,3))
femur_shaft = np.array([7.424,9.474,44.711]).reshape((1,3))
dent = np.array([5.018,9.058,47.535]).reshape((1,3))

viewer_landmark = view(selected_input_image,
     gradient_opacity=gradient_opacity,
     background=background,
     cmap=cmap,
     camera=camera,
     slicing_planes=True,
     ui_collapsed=True,
     opacity_gaussians=opacity_gaussians_landmark,
     point_set_sizes=[15,]*3,
     point_set_representations=['points',]*3,
     point_sets=[center_of_femur_head, femur_shaft, dent])
def on_select_input_viewer_landmark_update(change):
    viewer_landmark.image = selected_input_image
selected_input_callbacks.append(on_select_input_viewer_landmark_update)

head_center_widgets_top = [ColorPicker(description='Head Center ', concise=True, value='#d60000', disabled=True),
                           Button(description='From last click', tooltip='Set from last clicked slice point', icon='dot-circle-o'),]
head_center_widgets_bottom = [FloatText(value=4.945, description='x:'),
                              FloatText(value=9.225, description='y:'),
                              FloatText(value=46.011, description='z:'),]
def head_center_from_click(change):
    position = viewer_landmark.clicked_slice_point.position
    for i in range(3):
        head_center_widgets_bottom[i].value = position[i]
head_center_widgets_top[1].on_click(head_center_from_click)
head_center_box = VBox([HBox(head_center_widgets_top), HBox(head_center_widgets_bottom)])

shaft_widgets_top = [ColorPicker(description='Shaft', concise=True, value='#8c39ff', disabled=True),
                     Button(description='From last click', tooltip='Set from last clicked slice point', icon='dot-circle-o'),]
shaft_widgets_bottom = [FloatText(value=7.424, description='x:'),
                        FloatText(value=9.474, description='y:'),
                        FloatText(value=44.711, description='z:'),]
shaft_box = VBox([HBox(shaft_widgets_top), HBox(shaft_widgets_bottom)])

indent_widgets_top = [ColorPicker(description='Indent', concise=True, value='#018700', disabled=True),
                     Button(description='From last click', tooltip='Set from last clicked slice point', icon='dot-circle-o'),]
indent_widgets_bottom = [FloatText(value=5.018, description='x:'),
                         FloatText(value=9.058, description='y:'),
                         FloatText(value=47.535, description='z:'),]
indent_box = VBox([HBox(indent_widgets_top), HBox(indent_widgets_bottom)])

def update_positions():
    head_center_point = np.array([ft.value for ft in head_center_widgets_bottom]).reshape((1,3))
    shaft_point = np.array([ft.value for ft in shaft_widgets_bottom]).reshape((1,3))
    indent_point = np.array([ft.value for ft in indent_widgets_bottom]).reshape((1,3))
    viewer_landmark.point_sets = [head_center_point, shaft_point, indent_point]


def head_center_from_click(change):
    if viewer_landmark.clicked_slice_point is None:
        return
    position = viewer_landmark.clicked_slice_point.position
    for i in range(3):
        head_center_widgets_bottom[i].value = position[i]
    update_positions()
head_center_widgets_top[1].on_click(head_center_from_click)
def shaft_from_click(change):
    if viewer_landmark.clicked_slice_point is None:
        return
    position = viewer.clicked_slice_point.position
    for i in range(3):
        shaft_widgets_bottom[i].value = position[i]
    update_positions()
shaft_widgets_top[1].on_click(shaft_from_click)
def indent_from_click(change):
    if viewer_landmark.clicked_slice_point is None:
        return
    position = viewer_landmark.clicked_slice_point.position
    for i in range(3):
        indent_widgets_bottom[i].value = position[i]
    update_positions()
shaft_widgets_top[1].on_click(shaft_from_click)
for float_input in head_center_widgets_bottom + shaft_widgets_bottom + indent_widgets_bottom:
    float_input.observe(lambda x: update_positions(), 'value')

select_landmark_viewers = VBox([viewer_atlas, viewer_landmark])
position_widgets = VBox([head_center_box, shaft_box, indent_box])

select_landmark_widget = VBox([select_landmark_viewers, position_widgets])

# select_landmark_widget

In [7]:
# Landmark registration tab

landmark_registration_button = Button(description='Run registration')

landmark_registration_viewer = view(image=selected_input_image, **viewer_kwargs)
def on_select_input_landmark_registration(change):
    landmark_registration_viewer.image = selected_input_image
selected_input_callbacks.append(on_select_input_landmark_registration)

landmark_registration_widget = VBox([landmark_registration_button, landmark_registration_viewer])

def run_landmark_registration(change):
    image = selected_input_image
    label_image = itk.imread(segment_bones_path)
    landmark_registration = itk.LandmarkAtlasSegmentationFilter[type(image), type(label_image)].New()
    landmark_registration.SetInput(image)
    landmark_registration.SetInput(1, atlas)
    landmark_registration.SetInputLabels(label_image)
    landmark_registration.SetAtlasLabels(atlas_label)

    LandmarksType = itk.vector[itk.Point[itk.D, 3]]

    input_landmarks = LandmarksType()
    input_landmarks.push_back([ft.value for ft in head_center_widgets_bottom])
    input_landmarks.push_back([ft.value for ft in shaft_widgets_bottom])
    input_landmarks.push_back([ft.value for ft in indent_widgets_bottom])
    landmark_registration.SetInputLandmarks(input_landmarks)

    atlas_landmarks = LandmarksType()
    atlas_landmarks.push_back([4.072,11.347,43.816])
    atlas_landmarks.push_back([7.713,9.107,42.649])
    atlas_landmarks.push_back([3.787,11.172,45.173])
    landmark_registration.SetAtlasLandmarks(atlas_landmarks)
    landmark_registration_button.button_style = 'info'
    landmark_registration.Update()
    label_image = landmark_registration.GetOutput()
    viewer = view(image=selected_input_image, label_image=label_image, **viewer_kwargs)
    landmark_registration_widget.children = [landmark_registration_button, viewer]
    landmark_registration_button.button_style = 'success'
    
landmark_registration_button.on_click(run_landmark_registration)



In [8]:
tab_names = ['Select input',
             'Segment bones',
             'Select landmarks',
             'Landmark registration']
tab_children = [input_select_box,
                segment_bones_vbox,
                select_landmark_widget,
                landmark_registration_widget]

tab_widget = Tab()
tab_widget.children = tab_children
for ii, name in enumerate(tab_names):
    tab_widget.set_title(ii, name)
tab_widget

itkPointSetD3 not loaded from module ITKRegistrationCommon because of exception:
 module 'itk.ITKRegistrationCommonPython' has no attribute 'itkPointSetD3'
Template itk::LandmarkAtlasSegmentationFilter<itk::Image<signedshort,3>,itk::Image<signedshort,3>>
 already defined as <class 'itk.itkLandmarkAtlasSegmentationFilterPython.itkLandmarkAtlasSegmentationFilterISS3ISS3'>
 is redefined as <class 'itk.itkLandmarkAtlasSegmentationFilterPython.itkLandmarkAtlasSegmentationFilterISS3ISS3'>


In [43]:
# Deformation tab

input_masked_trabecular = itk.mask_negated_image_filter(image, mask_image=label_image, masking_value=2)
input_masked_cancellous = itk.mask_negated_image_filter(image, mask_image=label_image, masking_value=1)
input_masked = itk.add_image_filter(input_masked_trabecular, input_masked_cancellous)
del input_masked_trabecular
del input_masked_cancellous

In [44]:
atlas_masked = itk.mask_image_filter(atlas, mask_image=atlas_label, masking_value=0)

In [45]:
label_masked_trabecular = itk.mask_negated_image_filter(label_image, mask_image=label_image, masking_value=2)
label_masked_cancellous = itk.mask_negated_image_filter(label_image, mask_image=label_image, masking_value=1)
label_masked = itk.add_image_filter(label_masked_trabecular, label_masked_cancellous)
label_masked = itk.cast_image_filter(label_masked, ttype=(type(label_masked), itk.Image[itk.UC, 3]))
del label_masked_trabecular
del label_masked_cancellous

In [46]:
label_masked_map = itk.label_image_to_label_map_filter(label_masked)
cropped = itk.auto_crop_label_map_filter(label_masked_map, crop_border=[6,]*3)
label_masked_cropped = itk.label_map_to_label_image_filter(cropped)

In [51]:
input_masked_cropped = itk.extract_image_filter(input_masked,
                                                extraction_region=label_masked_cropped.GetLargestPossibleRegion())
view(input_masked_cropped,
     label_image=label_masked_cropped,
     cmap=cm.bone,
     camera=camera,
     gradient_opacity=1.0,
     background=(0.6,0.6,0.6),
     opacity_gaussians=opacity_gaussians
     )

Viewer(background=(0.6, 0.6, 0.6), camera=array([[-8.23076   , 16.761303  , 41.647186  ],
       [ 4.855     ,…

In [52]:
transform = landmark_registration.GetFinalTransform()
atlas_affine_transformed = itk.resample_image_filter(atlas_masked,
                                                     use_reference_image=True,
                                                     reference_image=input_masked_cropped,
                                                     transform=transform)

In [53]:
nearest_interpolator = itk.NearestNeighborInterpolateImageFunction.New(atlas_label)
atlas_labels_affine_transformed = itk.resample_image_filter(atlas_label,
                                                            use_reference_image=True,
                                                            reference_image=input_masked_cropped,
                                                            transform=transform,
                                                            interpolator=nearest_interpolator)

In [55]:
itk.imwrite(input_masked_cropped, file_name + '-input-masked-cropped.nrrd')
itk.imwrite(atlas_affine_transformed, file_name + '-atlas-affine-transformed.nrrd')
itk.imwrite(atlas_labels_affine_transformed, file_name + '-atlas-labels-affine-transformed.nrrd')
view(atlas_affine_transformed,
     label_image=atlas_labels_affine_transformed,
     cmap=cm.bone,
     camera=camera,
     gradient_opacity=1.0,
     background=(0.6,0.6,0.6),
     opacity_gaussians=opacity_gaussians
     )

Viewer(background=(0.6, 0.6, 0.6), camera=array([[-8.23076   , 16.761303  , 41.647186  ],
       [ 4.855     ,…

In [56]:
input_masked_cropped_f = itk.imread(file_name + '-input-masked-cropped.nrrd', itk.F)
atlas_affine_transformed_f = itk.imread(file_name + '-atlas-affine-transformed.nrrd', itk.F)
atlas_labels_affine_transformed_f = itk.imread(file_name + '-atlas-labels-affine-transformed.nrrd', itk.F)

In [57]:
%time registered_atlas, transform_parameter_object = itk.elastix_registration_method(input_masked_cropped_f, atlas_affine_transformed_f)

CPU times: user 3min 37s, sys: 5.79 s, total: 3min 43s
Wall time: 49.3 s


In [58]:
transformer = itk.TransformixFilter[type(atlas_labels_affine_transformed_f)].New()
transformer.SetMovingImage(atlas_labels_affine_transformed_f)
transformer.SetTransformParameterObject(transform_parameter_object)
transformer.Update()
transformed = transformer.GetOutput()
transformed_cast = itk.cast_image_filter(transformed, ttype=(type(transformed), itk.Image[itk.UC, 3]))

In [67]:
opacity_gaussians = [[{'position': 0.5,
   'height': 0.5636363636363637,
   'width': 0.5,
   'xBias': 0.026666666666666672,
   'yBias': 0}]]
registered = view(input_masked_cropped_f,
                  label_image=transformed_cast,
                  label_image_blend=0.7,
                  camera=camera,
                  gradient_opacity=1.0,
                  background=(0.6,0.6,0.6),
                  cmap=cm.bone,
                  opacity_gaussians=opacity_gaussians,
                  ui_collapsed=True,
                  )
registered

Viewer(background=(0.6, 0.6, 0.6), camera=array([[-8.23076   , 16.761303  , 41.647186  ],
       [ 4.855     ,…

In [69]:
weights = np.ones((10,), dtype=np.float32)*0.1
weights[5] = 1.0
registered.label_image_weights = weights

In [74]:
roi = itk.mask_negated_image_filter(transformed_cast, mask_image=transformed_cast, masking_value=5)
# view(label_image=roi)

In [75]:
morphometry_filter = itk.BoneMorphometryFeaturesFilter.New(input_masked_cropped_f)
morphometry_filter.SetMaskImage(roi)
morphometry_filter.Update()

In [76]:
print('BVTV', morphometry_filter.GetBVTV())
print('TbN', morphometry_filter.GetTbN())
print('TbTh', morphometry_filter.GetTbTh())
print('TbSp', morphometry_filter.GetTbSp())
print('BSBV', morphometry_filter.GetBSBV())

BVTV 0.7600745916651215
TbN 15.203703884426831
TbTh 0.049992725288715116
TbSp 0.015780720945284547
BSBV 40.00582061589391
